In [0]:
import pandas as pd
from spacy import displacy
import spacy

In [2]:
!git clone https://github.com/abasu644/dataanalysis.git

fatal: destination path 'dataanalysis' already exists and is not an empty directory.


In [0]:
import string
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
# Create our list of punctuation marks
punctuations = string.punctuation
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
from spacy.matcher import Matcher
from spacy.matcher import PhraseMatcher
from sklearn.model_selection import train_test_split
import numpy as np
def incidentFlag(ic):
    #print(ic)
    if type(ic) == float and np.isnan(ic):
        return 0
    else:
        return 1

def clean_text(text):
    # Removing spaces and converting text into lowercase
    return text.strip().lower()


In [0]:

# Custom transformer using spaCy
class AIOpsTextTransformer(TransformerMixin):
     
    def transform(self, X, **transform_params):
        # Cleaning Text
        X=[clean_text(text) for text in X]
        nlp = spacy.load("en_core_web_sm")
        stop_words = spacy.lang.en.stop_words.STOP_WORDS
        matcher = PhraseMatcher(nlp.vocab, attr="SHAPE")
        matcher.add("IP", None, nlp("127.0.0.1"), nlp("127.127.0.0"),nlp("10.59.100.122"))
        matcherTok = Matcher(nlp.vocab)
        matcherTok.add("URL",None,[{"LIKE_URL": True}])
        matcherTok.add("num", None, [{"LIKE_NUM": True}])
        matcherTok.add("endPat", None, [{"TEXT": {"REGEX": "%%%$"}}])
        #matcherTok.add("amp", None, [{"TEXT": {"REGEX": "[.]*[&!][.]*"}}])
        matcherTok.add("numb", None, [{"TEXT": {"REGEX": "[.]*[0-9\?\[\{\(\)][.]*"}}])
        #matcherTok.add("escape", None, [{"TEXT": {"REGEX": "(\x9B|\x1B\[)[0-?]*[ -\/]*[@-~]"}}])
        def custom_tokenizer(sentence):
          #print(sentence)
          rm_tk=[]
          # Get the IP
          doc = nlp(sentence)
          for match_id, start, end in matcher(doc):
              rm_tk.append(doc[start:end].text)
          for match_id, start, end in matcherTok(doc):
              rm_tk.append(doc[start:end].text)
          #print(rm_tk)
          tklp=[]
          tkl=[]
          for tk in doc:
              if tk.dep_=='punct':
                  continue
              elif tk.text.lower() in ['spendhq', '-', 'triggered',' ','edit','view']:
                  continue
              else:
                  tkl.append(tk.text)
                  tklp.append(tk.dep_)
          # Removing stop words
          mytokens = [ word for word in tkl if word not in stop_words and word not in punctuations ]
          mytokens=[ x for x in mytokens if x  not in rm_tk ]
          return mytokens
        finalStr=[]
        for row in X:
          finalStr.append(" ".join(custom_tokenizer(row)))
        return finalStr

    def fit(self, X, y=None, **fit_params):
        return self

    def get_params(self, deep=True):
        return {}
    
    



In [0]:
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

df=pd.read_csv("dataanalysis/data/datadog_spenhq_deduplicated_23incident.csv")
df=df.sort_values(by='date_happened') 
df['final'] = df.apply(lambda x: str(x['title'])+ " "+ str(x['text']+ " "+ str(x['tags'])),axis=1) # the features we want to analyze
X = df['final']
ylabels = df['case_id'].apply(lambda x : incidentFlag(x) )# the labels, or answers, we want to test against
X_train, X_test, y_train, y_test = train_test_split(X, ylabels, test_size=0.3,shuffle=False)
dataset={'train':(X_train,y_train),'test':(X_test,y_test)}
import pickle
with open('spendhq.data.pickle', 'wb') as output:
    pickle.dump(dataset, output)

In [22]:
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

import pickle

with open('spendhq.data.pickle', 'rb') as data:
    dataset = pickle.load(data)

X_train=dataset['train'][0]
y_train=dataset['train'][1]

X_test=dataset['test'][0]
y_test=dataset['test'][1]
import spacy
nlp = spacy.load('en_core_web_sm', disable=['ner', 'parser', 'tagger'])

def spacy_tokenizer(doc):
    return [x.orth_ for x in nlp(doc)]

classifier= RandomForestClassifier(n_estimators=100, max_depth=5,random_state=0)
bow_vector = CountVectorizer( tokenizer=spacy_tokenizer,ngram_range=(1,1))
# Create pipeline using Bag of Words
pipe = Pipeline([
                 ("aiops", AIOpsTextTransformer()),
                 ('vectorizer', bow_vector),
                 ('classifier', classifier)])

# model generation
pipe.fit(X_train,y_train)

# Predicting with a test dataset
predicted = pipe.predict(X_test)
print(X_test.shape)
# Model Accuracy
print("RandomForest Regression Accuracy:",metrics.accuracy_score(y_test, predicted))
print("RandomForest Regression Precision:",metrics.precision_score(y_test, predicted))
print("RandomForest Regression Recall:",metrics.recall_score(y_test, predicted))

(242,)
Logistic Regression Accuracy: 0.9752066115702479
Logistic Regression Precision: 0.3333333333333333
Logistic Regression Recall: 1.0


In [23]:
print(sum(y_test),len(y_test),sum(predicted))
a=pd.DataFrame(list(zip(y_test, predicted)),columns=['y','y_hat'])
a[(a.y_hat==1)]


3 242 9


,y,y_hat
69,1,1
70,0,1
76,0,1
77,1,1
78,1,1
214,0,1
216,0,1
223,0,1
224,0,1


In [0]:
metrics.f1_score(y_test, predicted)

In [0]:
df.shape